## Associate for every lap, the average weather data info

In [1]:
import fastf1 as ff1
import numpy as np
import pandas as pd
#from tqdm.notebook import tqdm  # Per la barra di progresso (opzionale)
from scipy.stats import circmean

# Select a driver

In [2]:
piloti_per_anno = {
    2015: ["HAM", "ROS", "VET", "RAI", "BOT", "MAS", "ALO", "BUT", "RIC", "KVY", "PER", "HUL", "GRO", "MAL", "SAI", "VER", "ERI", "NAS"],
    2016: ["HAM", "ROS", "VET", "RAI", "BOT", "MAS", "ALO", "BUT", "RIC", "VER", "PER", "HUL", "GRO", "GUT", "SAI", "KVY", "ERI", "NAS", "WEH", "OCO"],
    2017: ["VET", "HAM", "BOT", "RAI", "VER", "RIC", "PER", "OCO", "HUL", "SAI", "STR", "MAS", "ALO", "VAN", "GRO", "MAG", "ERI", "WEH"],
    2018: ["HAM", "VET", "RAI", "BOT", "VER", "RIC", "PER", "OCO", "HUL", "SAI", "LEC", "ERI", "GRO", "MAG", "ALO", "VAN", "STR", "SIR"],
    2019: ["HAM", "BOT", "VET", "LEC", "VER", "GAS", "ALB", "RIC", "HUL", "SAI", "PER", "STR", "RAI", "GIO", "GRO", "MAG", "KUB", "RUS"],
    2020: ["HAM", "BOT", "VER", "ALB", "LEC", "VET", "PER", "STR", "RIC", "OCO", "SAI", "NOR", "RAI", "GIO", "GRO", "MAG", "RUS", "LAT"],
    2021: ["VER", "HAM", "BOT", "PER", "LEC", "SAI", "RIC", "NOR", "GAS", "TSU", "OCO", "ALO", "VET", "STR", "RAI", "GIO", "RUS", "LAT", "MSC", "MAZ"],
    2022: ["VER", "PER", "LEC", "SAI", "HAM", "RUS", "NOR", "RIC", "OCO", "ALO", "GAS", "TSU", "VET", "STR", "BOT", "ZHO", "MSC", "MAG", "ALB", "LAT"],
    2023: ["VER", "PER", "HAM", "RUS", "LEC", "SAI", "NOR", "PIA", "ALO", "STR", "GAS", "OCO", "ALB", "SAR", "BOT", "ZHO", "MAG", "HUL"],
    2024: ["VER", "PER", "HAM", "RUS", "LEC", "SAI", "NOR", "PIA", "ALO", "STR", "GAS", "OCO", "ALB", "SAR", "BOT", "ZHO", "MAG", "HUL", "RIC", "TSU"]
}

gp_per_anno = {
    2015: ["Australia", "Malaysia", "China", "Bahrain", "Spain", "Monaco", "Canada", "Austria", "Great Britain", "Hungary", "Belgium", "Monza", "Singapore", "Japan", "Russia", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2016: ["Australia", "Bahrain", "China", "Russia", "Spain", "Monaco", "Canada", "Azerbaijan", "Austria", "Great Britain", "Hungary", "Germany", "Belgium", "Monza", "Singapore", "Malaysia", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2017: ["Australia", "China", "Bahrain", "Russia", "Spain", "Monaco", "Canada", "Azerbaijan", "Austria", "Great Britain", "Hungary", "Belgium", "Monza", "Singapore", "Malaysia", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2018: ["Australia", "Bahrain", "China", "Azerbaijan", "Spain", "Monaco", "Canada", "France", "Austria", "Great Britain", "Germany", "Hungary", "Belgium", "Monza", "Singapore", "Russia", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2019: ["Australia", "Bahrain", "China", "Azerbaijan", "Spain", "Monaco", "Canada", "France", "Austria", "Great Britain", "Germany", "Hungary", "Belgium", "Monza", "Singapore", "Russia", "Japan", "Mexico", "Austin", "Brazil", "Abu Dhabi"],
    2020: ["Austria", "Styria", "Hungary", "Great Britain", "70thAnniversary", "Spain", "Belgium", "Monza", "Tuscany", "Russia", "Eifel", "Portugal", "Imola", "Turkey", "Bahrain", "Sakhir", "Abu Dhabi"],
    2021: ["Bahrain", "Imola", "Portugal", "Spain", "Monaco", "Azerbaijan", "France", "Styria", "Austria", "Great Britain", "Hungary", "Belgium", "Zandvoort", "Monza", "Russia", "Turkey", "Austin", "Mexico", "Brazil", "Qatar", "Saudi Arabia", "Abu Dhabi"],
    2022: ["Bahrain", "Saudi Arabia", "Australia", "Imola", "Miami", "Spain", "Monaco", "Azerbaijan", "Canada", "Great Britain", "Austria", "France", "Hungary", "Belgium", "Zandvoort", "Monza", "Singapore", "Japan", "Austin", "Mexico", "Brazil", "Abu Dhabi"],
    2023: ["Bahrain", "Saudi Arabia", "Australia", "Azerbaijan", "Miami", "Monaco", "Spain", "Canada", "Austria", "Great Britain", "Hungary", "Belgium", "Zandvoort", "Monza", "Singapore", "Japan", "Qatar", "Austin", "Mexico", "Brazil", "LasVegas", "Abu Dhabi"],
    2024: ["Bahrain", "Saudi Arabia", "Australia", "Japan", "China", "Miami", "Imola", "Monaco", "Canada", "Spain", "Austria", "Great Britain", "Hungary", "Belgium", "Zandvoort", "Monza", "Azerbaijan", "Singapore", "Austin", "Mexico", "Brazil", "LasVegas", "Qatar", "Abu Dhabi"]
}
# Funzione per scegliere da un menu
def scegli_opzione(opzioni, messaggio):
    print(f"\n{messaggio}:")
    for i, opzione in enumerate(opzioni, 1):
        print(f"{i}. {opzione}")
    
    while True:
        try:
            scelta = int(input("Inserisci il numero: "))
            if 1 <= scelta <= len(opzioni):
                return opzioni[scelta - 1]
            print("Numero non valido.")
        except ValueError:
            print("Input non valido. Riprova.")

# --- Scelta anno ---
year = scegli_opzione(list(gp_per_anno.keys()), "Scegli l'anno")

# --- Scelta GP ---
gp = scegli_opzione(gp_per_anno[year], f"Scegli il GP del {year}")

# --- Scelta pilota (lista fissa per l'anno) ---
driver = scegli_opzione(piloti_per_anno[year], f"Scegli il pilota per il {year}")

# Risultato finale
print(f"\nHai selezionato: {driver} al GP di {gp} nel {year}!")


Scegli l'anno:
1. 2015
2. 2016
3. 2017
4. 2018
5. 2019
6. 2020
7. 2021
8. 2022
9. 2023
10. 2024



Scegli il GP del 2018:
1. Australia
2. Bahrain
3. China
4. Azerbaijan
5. Spain
6. Monaco
7. Canada
8. France
9. Austria
10. Great Britain
11. Germany
12. Hungary
13. Belgium
14. Monza
15. Singapore
16. Russia
17. Japan
18. Austin
19. Mexico
20. Brazil
21. Abu Dhabi

Scegli il pilota per il 2018:
1. HAM
2. VET
3. RAI
4. BOT
5. VER
6. RIC
7. PER
8. OCO
9. HUL
10. SAI
11. LEC
12. ERI
13. GRO
14. MAG
15. ALO
16. VAN
17. STR
18. SIR
Numero non valido.

Hai selezionato: ALO al GP di Abu Dhabi nel 2018!


In [ ]:
years = list(gp_per_anno.keys())

Available years: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [25]:
circuits_length = {
    'Melbourne': 5278,          # Albert Park Circuit
    'Shanghai': 5451,           # Shanghai International Circuit
    'Suzuka': 5807,             # Suzuka International Racing Course
    'Sakhir': 5412,             # Bahrain International Circuit (Bahrain è il paese, Sakhir la città)
    'Jeddah': 6174,             # Jeddah Corniche Circuit
    'Miami': 5412,              # Miami International Autodrome
    'Imola': 4909,              # Autodromo Enzo e Dino Ferrari
    'Monte Carlo': 3337,        # Circuit de Monaco (Monaco è il paese, Monte Carlo la zona)
    'Monaco': 3337,             # Circuit de Monaco (Monaco è il paese, Monte Carlo la zona)
    'Barcelona': 4657,          # Circuit de Barcelona-Catalunya
    'Montréal': 4361,           # Circuit Gilles Villeneuve
    'Spielberg': 4318,          # Red Bull Ring (Austria)
    'Silverstone': 5891,        # Silverstone Circuit
    'Spa-Francorchamps': 7004,  # Circuit de Spa-Francorchamps (Belgio)
    'Budapest': 4381,           # Hungaroring
    'Zandvoort': 4259,          # Circuit Zandvoort
    'Monza': 5793,              # Autodromo Nazionale Monza
    'Baku': 6003,               # Baku City Circuit
    'Singapore': 4940,          # Marina Bay Street Circuit
    'Austin': 5513,             # Circuit of the Americas
    'Mexico City': 4304,        # Autódromo Hermanos Rodríguez
    'São Paulo': 4309,          # Autódromo José Carlos Pace (Interlagos)
    'Las Vegas': 6201,          # Las Vegas Strip Circuit
    'Lusail': 5419,             # Losail International Circuit (Qatar)
    'Yas Marina': 5281,         # Yas Marina Circuit
    'Yas Island': 5281,         # Yas Marina Circuit
    'Le Castellet': 5842,       # Circuit Paul Ricard
    'Sochi': 5848,              # Sochi Autodrom
    'Portimão': 4653,           # Autódromo Internacional do Algarve
    'Istanbul': 5338,           # Istanbul Park
    'Nürburgring': 5148,        # Nürburgring
    'Hockenheim': 4574,         # Hockenheimring
    'Adelaide': 3780,           # Adelaide Street Circuit
    'Phoenix': 3770,            # Phoenix Street Circuit
    'Estoril': 4415,            # Autódromo do Estoril
    'Magny-Cours': 4411,        # Circuit de Nevers Magny-Cours
    'Indianapolis': 4180,       # Indianapolis Motor Speedway
    'Kuala Lumpur': 5543,       # Sepang International Circuit (Malaysia)
    'Valencia': 4005,           # Circuit Ricardo Tormo
    'Yeongam': 5603,            # Korean International Circuit
    'Jerez de la Frontera': 4423,  # Circuit de Jerez
    'Kyalami': 4300,            # Kyalami Grand Prix Circuit
    'Donington': 4023,          # Donington Park Circuit
    'Oyama': 4571,              # Fuji Speedway (città più vicina)
    'Mugello':5245              # Autodromo internazionale del Mugello
}

# Create a copy of driver dataframe

In [ ]:
session = ff1.get_session(2018, 'Monaco', 'R')
sessione = session.load()
weather_data = session.weather_data
driver = 'VAN'
driver_laps = session.laps.pick_drivers(driver)
#sessione presa, io prendo tutti i dati per la normalizzzazione prima, gli anni ci sono, basta fare il ciclo che ho gia fatto, poi quali altri dati devo normalizzare io, basta aggiungere il calcolo annuale e totale qui

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '5', '44', '7', '77', '31', '10', '27', '33', '55', '9', '11', '20', '2', '8', '35', '18', '16', '28', '14']


# Add data from weather df

In [ ]:
driver_laps['AirTemp'] = np.nan
driver_laps['Humidity'] = np.nan
driver_laps['Pressure'] = np.nan
driver_laps['Rainfall'] = np.nan
driver_laps['Track_Temp'] = np.nan
driver_laps['WindDirection'] = np.nan
driver_laps['WindSpeed'] = np.nan
driver_laps['Cumulative_Tire_Distance'] = np.nan
driver_laps['Driver_Meters'] = np.nan

#Creo il contatore partendo dalla prima riga
i = 0

# Get the circuit length in meters
location = session.event.Location
print(location)
circuit_length = circuits_length.get(location, 0)

driver_meters = 0

for i in range(len(driver_laps)):
    lap_number = driver_laps.iloc[i]['LapNumber']
    tire_life = driver_laps.iloc[i]['TyreLife']

    #Definisco la variabile lap_start_driver, quando inizia il giro in corso(i)
    lap_start_driver = driver_laps['LapStartTime'].iloc[i]
    #Definisco la variabile lap_finish_driver, quando finisce il giro in corso(i)
    lap_finish_driver = driver_laps['Time'].iloc[i]
    #Stampo i valori
    print('Il giro inizia : ', lap_start_driver)
    print('Il giro finisce a: ', lap_finish_driver)

    print('Il numero del giro è: ', lap_number)
    print('La vita della gomma è: ', tire_life)

    # Seleziona solo le righe con Time tra lap_start_driver e lap_finish_driver
    weather_first_lap = weather_data[
        (weather_data['Time'] >= lap_start_driver) & 
        (weather_data['Time'] <= lap_finish_driver)
    ]
    print(weather_first_lap)

    ###############################################################################

    j = 0
    temperatura = 0
    humidity = 0
    rain = False
    track_temp = 0
    pressure = 0
    wind_speed = 0
    wind_direction = 0
    for j in range(len(weather_first_lap)):

        #Calcolo la temperatura media
        temperatura = temperatura + weather_first_lap['AirTemp'].iloc[j]
        
        #Calcolo umidità media
        humidity = humidity + weather_first_lap['Humidity'].iloc[j]

        #Calocolo pioggia True/False
        if weather_first_lap['Rainfall'].iloc[j] == True:
            rain = True
        
        #Calculate AVG track Temp
        track_temp = track_temp + weather_first_lap['TrackTemp'].iloc[j]

        #Calculate AVG Pressure
        pressure = pressure + weather_first_lap['Pressure'].iloc[j]

        #Calculate AVG Wind Speed
        wind_speed = wind_speed + weather_first_lap['WindSpeed'].iloc[j]

        #Calculate AVG Wind Direction
        wind_direction = wind_direction + weather_first_lap['WindDirection'].iloc[j]


        j = j + 1

    
    air_temp_avg = temperatura / len(weather_first_lap)
    print('Temperatura_media ',air_temp_avg)
    humidity_avg = humidity / len(weather_first_lap)
    print('Umidità ',humidity_avg)
    print('Piove? ',rain)
    trackTemp_AVG = track_temp / len(weather_first_lap)
    print('TrackTemp AVG ', trackTemp_AVG)
    pressure_avg = pressure / len(weather_first_lap)
    print('Pressure AVG', pressure_avg)
    wind_speed_avg = wind_speed / len(weather_first_lap)
    print('Wind Speed AVG', wind_speed_avg)
    wind_direction_avg = wind_direction / len(weather_first_lap)
    print('Wind Direction AVG', wind_direction_avg)

    if(tire_life and tire_life > 0 and not tire_life == np.nan):

        cumulative_tire_distance = int(tire_life * circuit_length)
        driver_meters = int(lap_number * circuit_length)
        print(f'Cumulative Tire Distance = {cumulative_tire_distance}: {tire_life} * {circuit_length}')
        print(f'Driver Meters = {driver_meters}: {lap_number} * {circuit_length}')

    else:
        cumulative_tire_distance = 0
        driver_meters = 0

    driver_laps.iloc[i, driver_laps.columns.get_loc('AirTemp')] = air_temp_avg
    driver_laps.iloc[i, driver_laps.columns.get_loc('Humidity')] = humidity_avg
    driver_laps.iloc[i, driver_laps.columns.get_loc('Pressure')] = pressure_avg
    driver_laps.iloc[i, driver_laps.columns.get_loc('Rainfall')] = rain
    driver_laps.iloc[i, driver_laps.columns.get_loc('Track_Temp')] = trackTemp_AVG
    driver_laps.iloc[i, driver_laps.columns.get_loc('WindDirection')]  = wind_direction_avg
    driver_laps.iloc[i, driver_laps.columns.get_loc('WindSpeed')] = wind_speed_avg
    driver_laps.iloc[i, driver_laps.columns.get_loc('Cumulative_Tire_Distance')] = cumulative_tire_distance
    driver_laps.iloc[i, driver_laps.columns.get_loc('Driver_Meters')] = driver_meters


C:\Users\dilau\AppData\Local\Temp\ipykernel_13384\4153584129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps['AirTemp'] = np.nan
C:\Users\dilau\AppData\Local\Temp\ipykernel_13384\4153584129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_laps['Humidity'] = np.nan
C:\Users\dilau\AppData\Local\Temp\ipykernel_13384\4153584129.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

Monte Carlo
Il giro inizia :  0 days 00:06:47.319000
Il giro finisce a:  0 days 00:08:20.580000
Il numero del giro è:  1.0
La vita della gomma è:  nan
                    Time  AirTemp  Humidity  Pressure  Rainfall  TrackTemp  \
7 0 days 00:07:22.684000     25.6      51.6    1015.8     False       33.6   

   WindDirection  WindSpeed  
7            172        0.5  
Temperatura_media  25.6
Umidità  51.6
Piove?  False
TrackTemp AVG  33.6
Pressure AVG 1015.8
Wind Speed AVG 0.5
Wind Direction AVG 172.0
Il giro inizia :  0 days 00:08:20.580000
Il giro finisce a:  0 days 00:09:41.450000
Il numero del giro è:  2.0
La vita della gomma è:  1.0
                    Time  AirTemp  Humidity  Pressure  Rainfall  TrackTemp  \
8 0 days 00:08:22.702000     25.6      51.4    1015.6     False       33.7   
9 0 days 00:09:22.713000     25.8      50.6    1015.7      True       33.8   

   WindDirection  WindSpeed  
8            164        0.6  
9            222        0.8  
Temperatura_media  25.7000000000

Show df copy

In [28]:
driver_laps

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,IsAccurate,AirTemp,Humidity,Pressure,Rainfall,Track_Temp,WindDirection,WindSpeed,Cumulative_Tire_Distance,Driver_Meters
356,0 days 00:08:20.580000,VAN,2,0 days 00:01:33.076000,1.0,NaN,NaT,NaT,NaT,0 days 00:00:40.416000,...,False,25.60,51.60,1015.80,False,33.60,172.0,0.50,0.0,0.0
357,0 days 00:09:41.450000,VAN,2,0 days 00:01:20.870000,2.0,1.0,NaT,NaT,0 days 00:00:21.779000,0 days 00:00:37.633000,...,True,25.70,51.00,1015.65,True,33.75,193.0,0.70,3337.0,6674.0
358,0 days 00:11:01.457000,VAN,2,0 days 00:01:20.007000,3.0,1.0,NaT,NaT,0 days 00:00:21.422000,0 days 00:00:37.315000,...,True,26.00,50.70,1015.60,True,33.90,181.0,0.60,6674.0,10011.0
359,0 days 00:12:21.295000,VAN,2,0 days 00:01:19.838000,4.0,1.0,NaT,NaT,0 days 00:00:21.171000,0 days 00:00:37.271000,...,True,26.00,50.70,1015.60,True,33.90,181.0,0.60,10011.0,13348.0
360,0 days 00:13:41.075000,VAN,2,0 days 00:01:19.780000,5.0,1.0,NaT,NaT,0 days 00:00:21.136000,0 days 00:00:37.435000,...,True,26.30,49.45,1015.65,True,33.75,207.0,1.35,13348.0,16685.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0 days 01:44:33.481000,VAN,2,0 days 00:01:25.691000,73.0,3.0,0 days 01:43:08.478000,NaT,0 days 00:00:29.562000,0 days 00:00:35.866000,...,False,25.80,58.15,1015.20,False,34.35,173.5,1.35,3337.0,243601.0
429,0 days 01:45:50.775000,VAN,2,0 days 00:01:17.294000,74.0,3.0,NaT,NaT,0 days 00:00:20.816000,0 days 00:00:36.137000,...,True,25.70,58.20,1015.20,False,34.50,149.0,1.40,6674.0,246938.0
430,0 days 01:47:08.048000,VAN,2,0 days 00:01:17.273000,75.0,3.0,NaT,NaT,0 days 00:00:20.623000,0 days 00:00:36.106000,...,True,25.50,58.70,1015.20,False,33.80,173.0,1.50,10011.0,250275.0
431,0 days 01:48:24.912000,VAN,2,0 days 00:01:16.864000,76.0,3.0,NaT,NaT,0 days 00:00:20.203000,0 days 00:00:36.032000,...,True,25.45,59.15,1015.10,False,34.05,194.5,1.40,13348.0,253612.0


Generate csv from df

In [31]:
driver_laps.to_csv(f'driver_laps_{driver}_in_2018.csv', index=False)